In [1]:
import json
import os
import glob
import re
import difflib

import numpy as np
import pandas as pd

import re
from gensim.parsing.preprocessing import remove_stopwords, preprocess_string
import spacy
from gensim import corpora, models, similarities
import gensim
import nltk
import py_stringmatching as sm 
from py_stringmatching.similarity_measure import cosine as cos
import pyLDAvis
%matplotlib inline

import wikipedia
import wikipediaapi
from unidecode import unidecode

import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

#-------
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import time
from cleantext import clean

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.13) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
/usr/lib/python3/dist-packages/html5lib/_trie/_base.py:3: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Mapping


In [ ]:
def read_profile(files):

    # a function to read profile data from profile_files (brand and user)

    """
    [Name]   [Followers]   [Followees]   [Posts]   [URL]   [T/F (you can ignore this field ]   [Category]   [Bio]   [E-mail]   [Phone]   [Profile_pic]

    """


    profile_df = []

    #itarate through the profile files
    for file in files:

        try:


            head, username = os.path.split(file)  #get the filename /username  (teh file name is the username )
            #use pandas to read the file , map each field to its respective headers for eg: col1  represent teh field Name
            prof_info = pd.read_csv(file,delimiter='\t', usecols=[0,1,2,3,4,5,6,7,8,9,10], names=['Name', 'Followers',"Followees",'Posts','URL','T/F','Category','Bio','E-mail','Phone','Profile_pic'], header=None)
            prof_info['username'] = username #add the username to the dataframe
            profile_df.append(prof_info) # save the frame into a list
        except:
            pass




    return pd.concat(profile_df) #concat all the Dataframes and return a single Dataframe with all the profiles

In [ ]:

def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]






In [ ]:
def process__User_nodes(node):

    #helper function to extract usernames from  edge_media_to_tagged_user & edge_media_to_sponsor_user nodes
    #

    res = []
    try:

        for edge in node['edges']:

            try:
                res.append(edge["node"]["user"]["username"])
            except:
                pass
    except:
        pass

    return ",".join(res)


def get_caption(node):

    #helper function to get the caption text
    res = []
    try:
        for edge in node['edges']:
            try:
                res.append(edge["node"]["text"])
            except:
                pass
    except:
        pass

    return ",".join(res)



In [ ]:
def process_json(data):
    """

    a function to process the raw json data and normalize the data


    :param data:
    :return:
    """

    dataset = {}
    dataset['edge_media_to_tagged_user'] = process__User_nodes(data.get('edge_media_to_tagged_user',{})) # get a comma separated list of all tagged users
    dataset['edge_media_to_sponsor_user'] = process__User_nodes(data.get('edge_media_to_sponsor_user',{})) # get a comma separated list of all sponserd users
    dataset['edge_media_to_caption'] = get_caption(data.get('edge_media_to_caption',{})) # get the post caption text
    dataset['owner_full_name'] = data.get('owner',{}).get("full_name",None) # get post owners  full name if avilable, Defaults to None
    dataset['owner_username'] = data.get('owner',{}).get("username",None) # get post owners  username if avilable, Defaults to None
    dataset['owner_id'] = data.get('owner',{}).get("id",None) # get post owners id if avilable, Defaults to None
    dataset['edge_media_preview_like'] = data.get('edge_media_preview_like',{}).get("count",None)  # get total likes on the post if avilable, Defaults to None
    dataset['edge_media_to_comment'] = data.get('edge_media_to_comment',{}).get("count",None) # get the total comment count if avilable, Defaults to None
    dataset["Id"] = data.get("id",None) # get the post id if avilable, Defaults to None
    dataset["Is_ad"] = data.get("is_ad",False)  # get the is_ad value  if avilable, Defaults to False
    dataset["Is_video"] = data.get("is_video",False)  # get the is_video value if avilable, Defaults to False
    dataset["Location"] = data.get("location",None)  # get the location data if avilable, Defaults to None
    dataset["json_file"] = data.get("id","")+'.json'  # get the json_file name json_file = [postid].json
    return dataset


def process_post(chunk):
    """

    function to process the post.json files
    a list of postid.json file path is given as input


    :param chunk:
    :return:
    """

    #itarate through each postid.json file
    for post_file in chunk:

        post_data = json.loads(open(post_file,'r',encoding='utf-8-sig').read()) #read the json file into post_data variable
        try:
            yield process_json(post_data)
            # use process_json function to process the raw json data and yield the result from here
            # a genrater is used here becouse we dont want to keep the large data dict in memmory
        except Exception as e:
            print(e)

        post_data = None #memmory optimization





converting profile and brand files to csv files  for faster read - you only need to run this one time
once you have th file in you directory  you may not need to run this  again -> uncomment the block bellow to run it


In [ ]:
#get all the files in the users_brands_SPOD and users_influencers_SPOD folders
#read_profile function appends all the data and returns a Dataframe

profile_brands = read_profile(glob.glob('users_brands_SPOD//*')) # get
profile_influencers = read_profile(glob.glob('users_influencers_SPOD//*'))

#save the processed data to csv files for simplicity and time management
# read_profile is a heavey opration and must be avoided whenever we can
profile_brands.to_csv('profile_brands.csv',index=False)
profile_influencers.to_csv('profile_influencers.csv',index=False)

# you should keep this block commented out once you have full data in the csv file
# this block should only be uncommented if you have new profile data to add  and it should be commented back once you have the new csv







In [ ]:
profile_brands = pd.read_csv('profile_brands.csv').add_prefix('Brand_') # load the brands csv file and give the coloumns a 'Brand_' prefix to identify between brand and  influencer profiles
profile_brands['Brand_username'] = '@'+profile_brands['Brand_username'].str.lower() #add @ to the username
profile_influencers = pd.read_csv('profile_influencers.csv').add_prefix('Influencer_')# load the influencers csv file and give the coloumns a 'Influencer_' prefix to identify between brand and  influencer profiles

#brands_to_check = ['amazon','sephora','CALVIN KLEIN','Tiffany & Co.','Bumble']  #list of brand names you want to get query the posts for
#brands_to_check = [x.lower() for x in brands_to_check]

#brands_with_followers_between = profile_brands[profile_brands['Brand_Followers'].between(1500000,1600000)]
#brands_to_check = brands_with_followers_between['Brand_Name'].values.tolist() 




In [ ]:
profile_brands.index.name
profile_brands.index.name = 'brand_id'
profile_brands = profile_brands.reset_index(level=0)
profile_brands

In [ ]:
profile_influencers.index.name
profile_influencers.index.name = 'influencer_id'
profile_influencers

In [ ]:
profile_influencers = profile_influencers.reset_index(level=0)

In [ ]:
profile_influencers

In [ ]:
#def fuzzy_match(brands_to_check_list,name):
    #try:

        #name = str(name).lower() # convert the profile name to lowercase so that case sensitivity doesn't impact the matching
        #closest_match = difflib.get_close_matches(name, brands_to_check_list, 1, 0.9) # use  difflib to get the closest match  with match score greater than 0.9
        #if len(closest_match) == 0:
            #return None # return null if no match is found
        #else:
            #return closest_match[0] #return the first match
    #except:
        #return None # return null on error


In [ ]:
#profile_brands['Brand_Name_matched'] = profile_brands['Brand_Name'].apply(lambda x:  fuzzy_match(brands_to_check,x)) #keep the fuzzymatched  data in the 'Brand_Name_matched' column

In [ ]:
#profile_brands = profile_brands[profile_brands['Brand_Name_matched'].notnull()] # remove all rows with null values in the 'Brand_Name_matched' columns  so that only the matched values remain

In [ ]:
profile_brands = profile_brands[profile_brands['Brand_Followers'].between(1500000,1600000)]
profile_brands

In [ ]:
#use pandas to read the post_info.txt file , map each field to its respective headers for eg: col1  represent the field username
post_info = pd.read_csv('post_info.txt',delimiter='\t', usecols=[1,2,3], names=['username', 'Sponsorship_label',"json_file"], header=None)
# get all posts by  users specified in the 'usernames_to_check'  list
# keep only the posts where 'Sponsorship_label' equals '1'
post_info = post_info[(post_info['Sponsorship_label'] == 1)]


we dont need the above part if we are useing all_sponserd_posts.json file

In [ ]:
#read json files from json_file folder
#json_files = [f"json_file//{x.strip()}" for x in post_info['json_file'].tolist()]

#with open('all_sponserd_posts.json','w',encoding='utf-8-sig') as myjsonfile:
    #json.dump([x for x in process_post(json_files)],myjsonfile) #merge all the json files into one file for faster acces next time

# you should keep this block commented out once you have full data in the json file
# this block should only be uncommented if you have new post files to add  and it should be commented back once you have the all_sponserd_posts.json file


In [ ]:
#chunk_df = pd.DataFrame([x for x in process_post(json_files)]) # process those json files and load it into a dataframe # uncomment this line if you dont want to use all_sponserd_posts.json

chunk_df = pd.DataFrame(json.loads(open('all_sponserd_posts.json','r',encoding='utf-8-sig').read()))#use the all_sponserd_posts.json file for faster access



In [ ]:
chunk_df = chunk_df.merge(post_info,on="json_file",how='left')   # merge the new dataframe with post_info Dataframe based on the "json_file" coloumn present in both the dataframes


In [ ]:
chunk_df 

In [ ]:
def get_all_usernames_from_tokens(edge_media_to_caption_tokes):

    # check if token array is empty or not if empty  return null
    if len(edge_media_to_caption_tokes) == 0:
        return None


    matched_brands = []  #to hold matched brands

    #itarate through all the tokens
    for token in edge_media_to_caption_tokes:
        b = str(token).strip().lower() #convert each token to lowercase and strip whitespaces for a standerdized form
        if b.startswith('@'): # if a token starts with "@" we can  safely assume its a username
            matched_brands.append(token) # add the token to the matched list

    if len(matched_brands) == 0:
        # there is no matched brands then return None
        return None
    else:
        #remove duplicates and return matched list
        return list(set(matched_brands))

def tokenize_str(string):
    #split string by whitespace remove all special chars Except
    string = str(string).lower() # convert the text into lowercase for easier matching
    string = string.replace('#'," #").replace('\n'," \n") # replace all "#" values with " #" (whitespace + #) otherwise this would give a false match
    return re.sub("[^\w@ ]", "", string).split()  #split string by whitespace remove all special chars Except @

In [ ]:
chunk_df['edge_media_to_caption_tokes'] =  chunk_df['edge_media_to_caption'].apply(tokenize_str,lambda x:x)  # tokanize the edge_media_to_caption

In [ ]:

chunk_df['brand_matched'] = chunk_df['edge_media_to_caption_tokes'].apply(lambda x:get_all_usernames_from_tokens(x)) # call get_all_usernames_from_tokens function to get all usernames from caption





In [ ]:
chunk_df = chunk_df.explode('brand_matched')

In [ ]:
chunk_df.dropna(subset=['brand_matched'],inplace=True) #remove all rows where brand_matched is null
del chunk_df['edge_media_to_caption_tokes'] # delete edge_media_to_caption_tokes - > we dont need this anymore


chunk_df = chunk_df[chunk_df['brand_matched'].isin(profile_brands['Brand_username'])] #check if the matched brand is in the brands_to_check list


In [ ]:
# join the new dataframe with profile_brands Dataframe,  keep all records where 'brand_matched' from the new df  equals 'Brand_username' on the profile_brands Dataframe
chunk_df = chunk_df.merge(profile_brands,left_on='brand_matched',right_on='Brand_username',how='left')

# join the new dataframe with profile_influencers Dataframe,  keep all records where 'username' from the new df  equals 'Influencer_username' on the profile_influencers Dataframe
chunk_df = chunk_df.merge(profile_influencers,left_on='username',right_on='Influencer_username',how='left')

In [ ]:
chunk_df # print the final dataframe

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', -1)
chunk_df # print the final dataframe

In [ ]:
chunk_df.to_csv('final_results.csv',index=False)

In [ ]:
df = pd.read_csv('final_results.csv')

In [ ]:
df

In [ ]:
g = df.groupby(['brand_matched']).apply(lambda x: x.nlargest(20,['Influencer_Followers'])).reset_index(drop=True)

In [ ]:
df_final = g[['brand_id','Brand_Name','Brand_Followers','Brand_Followees','Brand_Posts','Brand_URL','Brand_Category','Id','edge_media_to_caption','edge_media_preview_like','edge_media_to_comment','influencer_id','Influencer_Name','Influencer_Followers','Influencer_Followees','Influencer_Posts','Influencer_Category']]
df_final

In [ ]:
df_final.rename(columns = {'Id':'post_id','edge_media_to_caption':'post_caption', 'edge_media_preview_like':'post_likes','edge_media_to_comment':'post_comments'}, inplace = True)
df_final

In [ ]:
df_final.to_csv('df_final.csv',index=False)

In [2]:
df_final = pd.read_csv('df_final.csv')

-------------------------------------------------------------------

### Create Brand Nodes

In [3]:
wikipedia.set_lang('en')
wikiApi = wikipediaapi.Wikipedia('en')

In [42]:
emoji_pattern  = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)

def remove_emoji(input_string):
    for ele in input_string:
        word = emoji_pattern.sub(r'', input_string) # No emoji
    return word 

In [43]:
def cleanListOfWords(listOfWords):
    listOfWords =  [remove_emoji(i).lower() for i in listOfWords if i != '']
    listOfWords = [re.sub('\S*@\S*\s?', '', word).lower() for word in listOfWords]
    listOfWords = [re.sub(r"[^a-zA-Z0-9 ]", " ", word) for word in listOfWords]
    listOfWords = [re.sub('\s+', ' ', word) for word in listOfWords]
    listOfWords = [re.sub("\'", "", word) for word in listOfWords]
    listOfWords = [re.sub(r'[^\w\s]', '', word) for word in listOfWords]
    listOfWords = [unidecode(word) for word in listOfWords]
    return listOfWords

<>:3: DeprecationWarning: invalid escape sequence \S
<>:5: DeprecationWarning: invalid escape sequence \s
<>:3: DeprecationWarning: invalid escape sequence \S
<>:5: DeprecationWarning: invalid escape sequence \s
/tmp/ipykernel_8287/1104507845.py:3: DeprecationWarning: invalid escape sequence \S
  listOfWords = [re.sub('\S*@\S*\s?', '', word).lower() for word in listOfWords]
/tmp/ipykernel_8287/1104507845.py:5: DeprecationWarning: invalid escape sequence \s
  listOfWords = [re.sub('\s+', ' ', word) for word in listOfWords]


In [103]:
brandNames = list(set(df_final["Brand_Name"]))
brandNames

['Nike Training Club Live',
 'Gymshark Women',
 'Pizza Hut 🍕',
 'Massimo Dutti',
 'hotmiamistyles',
 'Philadelphia Eagles',
 'Rogue Fitness',
 'QUAY AUSTRALIA',
 'Barneys New York',
 'Bottega Veneta',
 'RubyRose_Oficial',
 'ROXY',
 'TEZENiS',
 'adidas London',
 'Amazon',
 'Rashida Jones',
 'Official Kylie Jenner Shop',
 'Burger King',
 'Glossier',
 'Ludovica Valli',
 'Koton',
 'live lokai',
 'CREME PARA ESTRIAS',
 'Bergdorf Goodman',
 'FARSÁLI - Beauty with Benefits',
 'CELINE',
 'McClure Twins - Ava and Alexis',
 'Universal Studios Hollywood',
 'Universal Orlando Resort',
 'Moda',
 'dm-drogerie markt Deutschland',
 'The Body Shop Official',
 'Wish',
 'Shawn Johnson East',
 'OnePlus']

In [104]:
cleanedBrandNames = cleanListOfWords(brandNames)
cleanedBrandNames

['nike training club live',
 'gymshark women',
 'pizza hut ',
 'massimo dutti',
 'hotmiamistyles',
 'philadelphia eagles',
 'rogue fitness',
 'quay australia',
 'barneys new york',
 'bottega veneta',
 'rubyrose oficial',
 'roxy',
 'tezenis',
 'adidas london',
 'amazon',
 'rashida jones',
 'official kylie jenner shop',
 'burger king',
 'glossier',
 'ludovica valli',
 'koton',
 'live lokai',
 'creme para estrias',
 'bergdorf goodman',
 'fars li beauty with benefits',
 'celine',
 'mcclure twins ava and alexis',
 'universal studios hollywood',
 'universal orlando resort',
 'moda',
 'dm drogerie markt deutschland',
 'the body shop official',
 'wish',
 'shawn johnson east',
 'oneplus']

In [105]:
q3_token = sm.QgramTokenizer(qval=3)
cosine = cos.Cosine()
jaccard = sm.Jaccard()

In [106]:
phrasesList = []
for (idx, brand) in enumerate(cleanedBrandNames):
    #get rid of all unecessary words that could skew cosine similarity
    plainBrandName = brandNames[idx]
    phrase = '%20'.join([remove_stopwords(abst) for abst in brand.split(' ')])
    url = f'https://en.wikipedia.org/w/api.php?action=query&list=search&srsearch={phrase}&utf8=&format=json'
    jsonPhrase = pd.read_json(url, orient='records')
    
    scores = []
    
    for brandToToken in [brand + '(company)']:
        phraseToken = q3_token.tokenize(brandToToken)
        for result in jsonPhrase['query']['search']:            
            title = result['title'].lower()           
            q3TokenSample = q3_token.tokenize(title)
            cosScoreQ3 = cosine.get_sim_score(q3TokenSample, phraseToken)
            scores.append([result['title'], cosScoreQ3]) 
        
    if('roxy' in brand):
        scores = [["Quicksilver (company)", 1]]
    if('moda' == brand):
        scores = [["Modà", 1.1]]

    if(scores):
        maxScore = max(scores, key=lambda item:item[1])
        phrasesList.append([plainBrandName, maxScore[0]])    
    else:
        phrasesList.append([plainBrandName, ''])    

In [107]:
phrasesList

[['Nike Training Club Live', 'Nike, Inc.'],
 ['Gymshark Women', 'Gymshark'],
 ['Pizza Hut 🍕', 'Pizza Hut'],
 ['Massimo Dutti', 'Massimo Dutti'],
 ['hotmiamistyles', ''],
 ['Philadelphia Eagles', 'Philadelphia Eagles'],
 ['Rogue Fitness', 'Rogue Fitness'],
 ['QUAY AUSTRALIA', 'Quay (restaurant)'],
 ['Barneys New York', 'Barneys New York'],
 ['Bottega Veneta', 'Bottega Veneta'],
 ['RubyRose_Oficial', ''],
 ['ROXY', 'Quicksilver (company)'],
 ['TEZENiS', 'Sandro Veronesi (entrepreneur)'],
 ['adidas London', 'Adidas'],
 ['Amazon', 'Amazon (company)'],
 ['Rashida Jones', 'Rashida Jones'],
 ['Official Kylie Jenner Shop', 'Kendall Jenner'],
 ['Burger King', 'Burger King'],
 ['Glossier', 'Glossy ibis'],
 ['Ludovica Valli', 'Ludovica Torelli'],
 ['Koton', 'Koton (company)'],
 ['live lokai', 'List of Tales from the Crypt episodes'],
 ['CREME PARA ESTRIAS', ''],
 ['Bergdorf Goodman', 'Bergdorf Goodman'],
 ['FARSÁLI - Beauty with Benefits', 'Li Yundi'],
 ['CELINE', 'Celine (brand)'],
 ['McClure Tw

In [64]:
def lemmatization(texts, nlp, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [65]:
def findKeywords(brandName, listOfWords, numWords):
    wikiPage = cleanListOfWords(listOfWords)
    wikiPage = [word for word in wikiPage if word.lower() not in brandName.lower().split(" ")]
    
    absGram = [remove_stopwords(abst) for abst in wikiPage]
    absGramSplit = [remove_stopwords(abst).split(' ') for abst in wikiPage]
    
    bigram = gensim.models.Phrases(absGramSplit, min_count=4, threshold=1000) 
    trigram = gensim.models.Phrases(bigram[absGramSplit], threshold=100)  

    # Faster way to get a sentence clubbed as a trigram/bigram
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram_mod = gensim.models.phrases.Phraser(trigram)

    def make_bigrams(texts):
        return [bigram_mod[doc] for doc in texts]

    def make_trigrams(texts):
        return [trigram_mod[bigram_mod[doc]] for doc in texts]

    data_words_trigrams = make_trigrams(absGramSplit)

    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

    tri_lemmatized = lemmatization(data_words_trigrams, nlp, allowed_postags=['NOUN', 'ADJ', 'VERB'])

    id2word = corpora.Dictionary(tri_lemmatized)
    texts = tri_lemmatized
    corpus = [id2word.doc2bow(text) for text in texts]

    allTopics = []
    ldaModel = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=numWords, 
                                           random_state=133,
                                           update_every=10,
                                           chunksize=len(wikiPage),
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=False)           

    doc_lda = ldaModel[corpus]

    topics = []
    
    for idx, topic in ldaModel.show_topics(formatted=False, num_words= numWords):
        topics.extend([w[0] for w in topic])
    
    return topics

In [108]:
keyWordsList = []
for brand in phrasesList:
    topicsOfTopics = []
    if(brand[1] != ''):
        wikiTitle = brand[1]
        wikiPage = wikipedia.summary(title = wikiTitle, auto_suggest=False).split(' ')
        topics = findKeywords(brand[0], wikiPage, 10);
        topicsOfTopics = findKeywords(brand[0], topics, 2)
    
    keyWordsList.append([brand[0], topicsOfTopics])
    
print(keyWordsList)

/home/kateolaughlin/.local/lib/python3.8/site-packages/spacy/language.py:1895: UserWarning: [W123] Argument disable with value ['parser', 'ner'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(


[['Nike Training Club Live', ['brand', 'air', 'company', 'equipment']], ['Gymshark Women', ['retailer', 'retail', 'fitness', 'company']], ['Pizza Hut 🍕', ['serve', 'location', 'chain', 'large']], ['Massimo Dutti', ['multinational', 'luxury', 'pronunciation', 'spanish']], ['hotmiamistyles', []], ['Philadelphia Eagles', ['football', 'appear', 'team', 'rivalry']], ['Rogue Fitness', ['gym', 'base', 'equipment', 'box']], ['QUAY AUSTRALIA', ['award', 'run', 'appear', 'restaurant']], ['Barneys New York', ['brand', 'avenue', 'authentic', 'fifth']], ['Bottega Veneta', ['branding', 'license', 'accessory', 'jewelry']], ['RubyRose_Oficial', []], ['ROXY', ['quicksilver', 'gambling', 'amusement', 'machine']], ['TEZENiS', ['net', 'sandro', 'worth', 'veronesi']], ['adidas London', ['large', 'shoe', 'dassler', 'spike']], ['Amazon', ['stream', 'refer', 'online', 'large']], ['Rashida Jones', ['series', 'hot', 'comedy', 'film']], ['Official Kylie Jenner Shop', ['list', 'fame', 'model', 'campaign']], ['Bur

-----------------------

### Create Nodes and Relationships

Create Brand Nodes

In [109]:
nodes = []
for word in keyWordsList:
    brand = df_final[df_final['Brand_Name'] == word[0]].iloc[0]
    nodes.append([brand['brand_id'], "Brand", brand['Brand_Name'], word[1], brand['Brand_Followers'], \
                  brand['Brand_Followees'], brand['Brand_Posts'], brand["Brand_URL"], brand["Brand_Category"]])
        
brandNodeDf = pd.DataFrame(nodes, columns=['brand_ID', ':LABEL','Brand_Name','Keywords', "Number_Of_Followers", "Number_Of_Followees", "Number_Of_Posts", "Brand_Url", "Categories"])

In [111]:
brandNodeDf.head()

,brand_ID,:LABEL,Brand_Name,Keywords,Number_Of_Followers,Number_Of_Followees,Number_Of_Posts,Brand_Url,Categories
0,19343,Brand,Nike Training Club Live,"[brand, air, company, equipment]",1570912.0,166.0,317.0,https://swoo.sh/2K7uhZs,NaN
1,16013,Brand,Gymshark Women,"[retailer, retail, fitness, company]",1503127.0,47.0,1917.0,http://gym.sh/AllYouNeedToKnowToronto,Personal Goods & General Merchandise Stores
2,8209,Brand,Pizza Hut 🍕,"[serve, location, chain, large]",1510972.0,371.0,1444.0,http://www.PizzaHut.com/NFL,Restaurants
3,24856,Brand,Massimo Dutti,"[multinational, luxury, pronunciation, spanish]",1576480.0,323.0,3002.0,http://mdutti.me/NorthWinds,Personal Goods & General Merchandise Stores
4,2836,Brand,hotmiamistyles,[],1597934.0,1059.0,10842.0,http://www.HotMiamiStyles.com/,Personal Goods & General Merchandise Stores


Create Post Nodes

In [112]:
postNodeList = []
influNodeList = []
for index, row in df_final.iterrows():
    postNodeList.append([row['post_id'], 'Post', row['post_caption'], row['post_likes'], row['post_comments']])
    if row['Influencer_Name'] not in [inf[2] for inf in influNodeList]:
        influNodeList.append([row['influencer_id'], 'Influencer', row['Influencer_Name'], \
                              row['Influencer_Followers'], row['Influencer_Followees'], row['Influencer_Posts'], row['Influencer_Category']])
    
postNodesDf = pd.DataFrame(postNodeList, columns=['post_ID', 'LABEL', 'Caption', 'Number_of_Likes', 'Number_of_Comments'])
influNodesDf = pd.DataFrame(influNodeList, columns=['influencerID', 'LABEL', 'Influencer_Name', "Number_Of_Followers", "Number_of_Followees", \
                                                   "Number_Of_Posts", "Influencer_Category"])
postNodesDf.head()

,post_ID,LABEL,Caption,Number_of_Likes,Number_of_Comments
0,1877001634463122445,Post,Eu e meu chamego de sempre passando pelo seu f...,46572,236
1,1891646163346665904,Post,"Minha cara quando falam: ""Lore, tem sobremesa""...",54989,286
2,1916289352699275991,Post,Tô de olho em você que ainda não seguiu minha ...,67286,400
3,1936582221779537401,Post,Pq se nao tiver essa cara vocês sabem que não ...,61472,370
4,1961799656992243034,Post,Minha cara para quem fala que ainda não compro...,38757,289


In [113]:
influNodesDf.head()

,influencerID,LABEL,Influencer_Name,Number_Of_Followers,Number_of_Followees,Number_Of_Posts,Influencer_Category
0,378335,Influencer,Lorena Improta,5074442.0,1966,5127.0,Creators & Celebrities
1,51458,Influencer,Gabi Brandt,1782185.0,302,1803.0,Creators & Celebrities
2,372578,Influencer,CREME PARA ESTRIAS,555818.0,1951,2045.0,Lifestyle Services
3,56832,Influencer,Priscila Simões,389510.0,750,1337.0,Creators & Celebrities
4,4635,Influencer,MORGANA SANTANA,256163.0,509,2029.0,Creators & Celebrities


In [117]:
df_final.iloc[index]

brand_id                                                             1279
Brand_Name                                               Bergdorf Goodman
Brand_Followers                                                 1529651.0
Brand_Followees                                                     878.0
Brand_Posts                                                        7668.0
Brand_URL                                     http://like2b.uy/bergdorfs/
Brand_Category                Personal Goods & General Merchandise Stores
post_id                                               1238009176605342650
post_caption            What's better than flowers and Eau de Parfum f...
post_likes                                                           1509
post_comments                                                          34
influencer_id                                                        8176
Influencer_Name                                                       NaN
Influencer_Followers                  

In [126]:
relbrandtopost = []
for index, row in df_final.iterrows():
    postId = postNodesDf[(postNodesDf['Caption'] == row['post_caption']) \
                         & (postNodesDf['Number_of_Likes'] == row['post_likes'])]['post_ID'].iloc[0]
    brandId = brandNodeDf[brandNodeDf['Brand_Name'] == row['Brand_Name']]['brand_ID'].iloc[0]
    relbrandtopost.append([brandId, "SPONSORED", postId])
edge_brandtopost = pd.DataFrame(relbrandtopost, columns=['brand_ID', 'TYPE', 'post_ID'])
edge_brandtopost.head()

,brand_ID,TYPE,post_ID
0,3209,SPONSORED,1877001634463122445
1,3209,SPONSORED,1891646163346665904
2,3209,SPONSORED,1916289352699275991
3,3209,SPONSORED,1936582221779537401
4,3209,SPONSORED,1961799656992243034


In [128]:
relinfltopost = []
for index, row in df_final.iterrows():
    if(str(row['Influencer_Name']) != 'nan'):
        postId = postNodesDf[(postNodesDf['Caption'] == row['post_caption']) \
                             & (postNodesDf['Number_of_Likes'] == row['post_likes'])]['post_ID'].iloc[0]
        influId = influNodesDf[influNodesDf['Influencer_Name'] == row['Influencer_Name']]['influencerID'].iloc[0]
        relinfltopost.append([influId, "POSTED", postId])
edge_relinfltopost = pd.DataFrame(relinfltopost, columns=['influencer_ID', 'TYPE', 'post_ID'])
edge_relinfltopost.head()

,influencer_ID,TYPE,post_ID
0,378335,POSTED,1877001634463122445
1,378335,POSTED,1891646163346665904
2,378335,POSTED,1916289352699275991
3,378335,POSTED,1936582221779537401
4,378335,POSTED,1961799656992243034


In [ ]:
influNodesDf.to_csv("InfluencerNodes.csv", index=False)
postNodesDf.to_csv("PostNodes.csv", index=False)
brandNodeDf.to_csv("BrandNodes.csv", index=False)
edge_relinfltopost.to_csv("Edges_Infl.csv", index=False)
edge_brandtopost.to_csv("Edges_Brand.csv", index=False)
#relationList.to_csv("files/relationList.csv", index=False)

In [ ]:
#./bin/neo4j-admin import --force --multiline-fields=true --nodes=./import/1_node.csv --relationships=./import/1_rel.csv

### Famous Person Webscrape Dataset

In [203]:
df_influencer = df_final["Influencer_Name"]
df_influencer

0                     Lorena Improta
1                     Lorena Improta
2                     Lorena Improta
3                     Lorena Improta
4                     Lorena Improta
                   ...              
437    Tiffany 🌹 - Beauty Influencer
438                    🌸 P I N V R 🌸
439                        Annie Cho
440          Hi y’all, I’m Brandy G!
441                            Mandy
Name: Influencer_Name, Length: 442, dtype: object

In [204]:
influencer_names = df_influencer.unique()

In [209]:
#remove emoji symbols from their names
emoji_pattern  = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
def remove_emoji(input_string):
    return emoji_pattern.sub(r'', input_string ) # No emoji

no_emoji =  [remove_emoji(i) if str(i) != 'nan' else i for i in influencer_names ]

print(no_emoji)

['Lorena Improta', 'Gabi Brandt', 'CREME PARA ESTRIAS', 'Priscila Simões', 'MORGANA SANTANA', 'F2Freestylers', 'Tom Daley', 'bradleysimmonds', 'TASHA GREEN', 'N S', 'Sophie Hellyer', 'Khloé', 'Vanessa Hudgens', 'Priyanka Chopra', 'Adriana Lima', 'Jessica Biel', 'Nia Sioux', 'Russell Wilson', 'Olivia Culpo', 'Dr. Holly Hatcher-Frazier', 'Kaycee Rice', 'Peter McKinnon', 'Sydney Leroux Dwyer', 'sung kang', 'Meghan King Edmonds', 'bri emery / designlovefest', 'Emily Luciano', 'Claire Godard', 'Farrah Davidson', 'KINDNESS • BEAUTY • LIFESTYLE', 'Franko Dean', 'THANIA | Fashion & Beauty Blog', 'The Perennial Style', 'Victoria Hui', 'Raysa Garcia', 'Mr.kool //Austin ™', 'TheSequinHanger', 'LaTonya', 'Carolyn', '| San Diego Blogger', 'T™ ', 'IDM', 'fᎪshᏁᏟuᏒᎥᎾus: #Fashion #Beauty', 'Opal Stewart', 'Alexandre | الكساندر', 'Amlul.com', 'ManhattanGirl', 'Tina Craig', 'THELIMITDOESNOTEXIST', 'yael steren', 'All The Pretty Birds', 'By Sophie and Charlotte ', nan, 'ELLEN COLE', 'SAVAGE IS THE KING ',

In [214]:
#clean up punctuation from list of influencer names

def remove_punc(string):
    punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~•'''
    for ele in string:  
        if ele in punc:  
            string = string.replace(ele, " ") 
    return string
 
no_punc_name = [remove_punc(i) if str(i) != 'nan' else i for i in no_emoji]
no_punc_name = [i.strip() if str(i) != 'nan' else i  for i in no_punc_name]

print(no_punc_name[:5]) # cleaned list and spaces stripped

['Lorena Improta', 'Gabi Brandt', 'CREME PARA ESTRIAS', 'Priscila Simões', 'MORGANA SANTANA']


<>:4: DeprecationWarning: invalid escape sequence \,
<>:4: DeprecationWarning: invalid escape sequence \,
/tmp/ipykernel_8287/235454804.py:4: DeprecationWarning: invalid escape sequence \,
  punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~•'''


In [216]:
clean_parens_text = [re.sub("[\(\[].*?[\)\]]", "", x)if str(x) != 'nan' else x for x in no_punc_name ]

clean_parens_text[:5]

<>:1: DeprecationWarning: invalid escape sequence \(
<>:1: DeprecationWarning: invalid escape sequence \(
/tmp/ipykernel_8287/4048533316.py:1: DeprecationWarning: invalid escape sequence \(
  clean_parens_text = [re.sub("[\(\[].*?[\)\]]", "", x)if str(x) != 'nan' else x for x in no_punc_name ]


['Lorena Improta',
 'Gabi Brandt',
 'CREME PARA ESTRIAS',
 'Priscila Simões',
 'MORGANA SANTANA']

In [218]:
#clean up prefix titles
titles = ("MR","DR","MRS","PROF","MS")
ptrn = re.compile(fr"^({'|'.join(titles)})\.?\s+", flags=re.I)
clean_names = [ptrn.sub("", i) if str(i) != 'nan' else i for i in clean_parens_text]
print(clean_names[:5])


['Lorena Improta', 'Gabi Brandt', 'CREME PARA ESTRIAS', 'Priscila Simões', 'MORGANA SANTANA']


In [219]:
len(clean_names)

279

#### Scrape Famous person attributes data if it exists

In [220]:
name_list = []
day_list = []
year_list = []
age_list = []
birth_list = []


options = Options()
DRIVER_PATH = '/Users/squach/Downloads/chromedriver'
driver = webdriver.Chrome(options=options, executable_path=DRIVER_PATH)


# options.headless = True
options.add_argument("--window-size=1920,1200")
action = webdriver.ActionChains(driver)

xpath_name = '/html/body/div[1]/div[1]/div/div/div[2]/div/div/div[1]/div/div[1]/h1'
xpath_date =  '/html/body/div[1]/div[1]/div/div/div[2]/div/div/div[2]/div/div[1]/div/div[1]/div/a[1]'
xpath_year = '/html/body/div[1]/div[1]/div/div/div[2]/div/div/div[2]/div/div[1]/div/div[1]/div/a[2]'
xpath_age = '/html/body/div[1]/div[1]/div/div/div[2]/div/div/div[2]/div/div[1]/div/div[3]/div/a'
birthplace_path = '/html/body/div[1]/div[1]/div/div/div[2]/div/div/div[2]/div/div[1]/div/div[2]/div'

for names in clean_names:
    if str(names) != 'nan' and names.strip() != '':
        driver.get("https:www.famousbirthdays.com")

        inputElement = driver.find_element("id","main-search")
        inputElement.send_keys(names)

        #print("looking at this name", names)

        inputElement.send_keys(Keys.ENTER)

        #wait 4.5 seconds so the site doesn't ask to confirm that i'm not a bot
        time.sleep(4.5) 

        try:
            name = driver.find_element("xpath",xpath_name).text
            date = driver.find_element("xpath",xpath_date).text
            year = driver.find_element("xpath",xpath_year).text
            age = driver.find_element("xpath",xpath_age).text
            birthplace = driver.find_element("xpath",birthplace_path).text

            name_list.append(name)
            day_list.append(date)
            year_list.append(year)
            age_list.append(age)
            birth_list.append(birthplace)
        except:
            name_list.append(names)
            day_list.append('n/a')
            year_list.append('n/a')
            age_list.append('n/a')
            birth_list.append('n/a')
    else:
        name_list.append(names)
        day_list.append('n/a')
        year_list.append('n/a')
        age_list.append('n/a')
        birth_list.append('n/a')
        
        
        driver.back()
    
    
    driver.back()

driver.close()
driver.quit()

/tmp/ipykernel_8287/2845466619.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(options=options, executable_path=DRIVER_PATH)


In [221]:
# it was easier to grab the Xpath to all the information under the Birthplace div class an just clean it up
# afterwards. Some only had the country listed, and for the City, States located separately and end up 
# combining it afterwards. 

birth_list_clean = [s.replace('BIRTHPLACE\n', '') for s in birth_list]
print(birth_list_clean[:10])

['Brazil', 'Sao Jose do Rio Preto, Brazil', 'n/a', 'Brazil', 'Brazil', 'n/a', 'Plymouth, England', 'England', 'England', 'n/a']


In [222]:
influencer_about_df = pd.DataFrame({'Unique_Influencer_Name':influencer_names})

In [223]:
# influencer_about_df

influencer_about_df['Clean_Name'] = name_list
influencer_about_df['Birthdate'] = day_list
influencer_about_df['Birth_Year'] = year_list
influencer_about_df['Age'] = age_list
influencer_about_df['Birthplace'] = birth_list_clean

In [224]:
influencer_about_df.to_csv("Influencer_Info_Scrape.csv", index=False)

In [225]:
influencer_about_df

,Unique_Influencer_Name,Clean_Name,Birthdate,Birth_Year,Age,Birthplace
0,Lorena Improta,Lorena Improta,September 1,1993,29 years old,Brazil
1,Gabi Brandt,Gabi Brandt,January 15,1996,26 years old,"Sao Jose do Rio Preto, Brazil"
2,CREME PARA ESTRIAS,CREME PARA ESTRIAS,n/a,n/a,n/a,n/a
3,Priscila Simões,Priscila Simões,September 18,1992,30 years old,Brazil
4,MORGANA SANTANA,Morgana Santana,June 9,1998,24 years old,Brazil
...,...,...,...,...,...,...
274,Brittney Mahler,Brittney Mahler,n/a,n/a,n/a,n/a
275,Tiffany 🌹 - Beauty Influencer,Tiffany Beauty Influencer,n/a,n/a,n/a,n/a
276,Annie Cho,Annie Cho,n/a,n/a,n/a,n/a
277,"Hi y’all, I’m Brandy G!",Hi y’all I’m Brandy G,n/a,n/a,n/a,n/a


In [226]:
influNodesDf

,influencerID,LABEL,Influencer_Name,Number_Of_Followers,Number_of_Followees,Number_Of_Posts,Influencer_Category
0,378335,Influencer,Lorena Improta,5074442.0,1966,5127.0,Creators & Celebrities
1,51458,Influencer,Gabi Brandt,1782185.0,302,1803.0,Creators & Celebrities
2,372578,Influencer,CREME PARA ESTRIAS,555818.0,1951,2045.0,Lifestyle Services
3,56832,Influencer,Priscila Simões,389510.0,750,1337.0,Creators & Celebrities
4,4635,Influencer,MORGANA SANTANA,256163.0,509,2029.0,Creators & Celebrities
...,...,...,...,...,...,...,...
274,175642,Influencer,Brittney Mahler,21308.0,2343,229.0,Creators & Celebrities
275,7392,Influencer,Tiffany 🌹 - Beauty Influencer,19260.0,6305,184.0,Creators & Celebrities
276,376151,Influencer,Annie Cho,16072.0,1825,531.0,Creators & Celebrities
277,50475,Influencer,"Hi y’all, I’m Brandy G!",15444.0,1565,1088.0,Creators & Celebrities


In [227]:
influNodesFinalDf = pd.merge(influNodesDf, influencer_about_df, how='inner', \
                             left_on = 'Influencer_Name', right_on = 'Unique_Influencer_Name')
influNodesFinalDf
influNodesFinalDf.head()

,influencerID,LABEL,Influencer_Name,Number_Of_Followers,Number_of_Followees,Number_Of_Posts,Influencer_Category,Unique_Influencer_Name,Clean_Name,Birthdate,Birth_Year,Age,Birthplace
0,378335,Influencer,Lorena Improta,5074442.0,1966,5127.0,Creators & Celebrities,Lorena Improta,Lorena Improta,September 1,1993,29 years old,Brazil
1,51458,Influencer,Gabi Brandt,1782185.0,302,1803.0,Creators & Celebrities,Gabi Brandt,Gabi Brandt,January 15,1996,26 years old,"Sao Jose do Rio Preto, Brazil"
2,372578,Influencer,CREME PARA ESTRIAS,555818.0,1951,2045.0,Lifestyle Services,CREME PARA ESTRIAS,CREME PARA ESTRIAS,n/a,n/a,n/a,n/a
3,56832,Influencer,Priscila Simões,389510.0,750,1337.0,Creators & Celebrities,Priscila Simões,Priscila Simões,September 18,1992,30 years old,Brazil
4,4635,Influencer,MORGANA SANTANA,256163.0,509,2029.0,Creators & Celebrities,MORGANA SANTANA,Morgana Santana,June 9,1998,24 years old,Brazil


In [ ]:
influNodesDf.to_csv("InfluencerNodes.csv", index=False)
postNodesDf.to_csv("PostNodes.csv", index=False)
brandNodeDf.to_csv("BrandNodes.csv", index=False)
edge_relinfltopost.to_csv("Edges_Infl.csv", index=False)
edge_brandtopost.to_csv("Edges_Brand.csv", index=False)
#relationList.to_csv("files/relationList.csv", index=False)